# AWS Bedrock LLMManager Multimodal Content Demo

This notebook demonstrates how to use the extended multimodal content capabilities of the LLMManager class to interact with AWS Bedrock's Converse API.

## Setup

Let's first import the necessary modules and initialize the LLMManager.

In [ ]:
import sys
import os
from typing import Dict, List, Any, Optional, Tuple

# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import from our package
from src.LLMManager import LLMManager
from src.content import ContentBuilder, TextContent, ImageContent, DocumentContent, VideoContent
from src.ConverseFieldConstants import Fields, Roles

## Initialize LLMManager

In [ ]:
# Replace with your AWS CLI profile name if needed
aws_profile = "default"

# Initialize LLMManager with AWS profile
llm_manager = LLMManager(
    profile_name=aws_profile,
    regions=["us-east-1", "us-west-2", "eu-west-1"],  # Try multiple regions
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"]  # Models with multimodal capabilities
)

## Basic Usage: Single Content Types

### 1. Text Message (Original Method)

In [ ]:
# Create a simple text message using the original method
text_message = llm_manager.create_text_message("What are the planets in our solar system?")

# Send the message to the model
response = llm_manager.converse(messages=[text_message])
print(response.get_content_text())

### 2. Single Image Message (Original Method)

In [ ]:
# Load an image
with open("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg", "rb") as f:
    image_bytes = f.read()

# Create an image message with text using the original method
image_message = llm_manager.create_image_message(
    image_bytes=image_bytes,
    text="What is this famous landmark?",
    image_format="jpeg"
)

# Send the message to the model
response = llm_manager.converse(messages=[image_message])
print(response.get_content_text())

### 3. Document Message (New Method)

In [ ]:
# Load a PDF document (you'll need to provide your own PDF)
# with open("path/to/your/document.pdf", "rb") as f:
#     document_bytes = f.read()

# Uncomment when you have a document to test with:
# document_message = llm_manager.create_document_message(
#     document_bytes=document_bytes,
#     document_name="example_document.pdf",
#     document_format="pdf",
#     text="Please summarize this document."
# )

# response = llm_manager.converse(messages=[document_message])
# print(response.get_content_text())

## Advanced Usage: Multiple Content Items in One Message

### 1. Multiple Images Message

In [ ]:
# Load multiple images
with open("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg", "rb") as f1:
    image1_bytes = f1.read()
    
# For demo purposes, using the same image twice
# In a real application, you'd use different images
image2_bytes = image1_bytes

# Create a message with multiple images
image_data_list = [
    (image1_bytes, "jpeg"),
    (image2_bytes, "jpeg")
]

multi_image_message = llm_manager.create_multi_image_message(
    image_data_list=image_data_list,
    text="What are the similarities between these structures?"
)

# Send the message to the model
# Note: This will only work if your model supports multiple images in one message
# response = llm_manager.converse(messages=[multi_image_message])
# print(response.get_content_text())

### 2. Multiple File Types Message

In [ ]:
# Load various file types
with open("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg", "rb") as f:
    image_bytes = f.read()

# For a real application, you'd have actual document bytes
# document_bytes = b"Sample document content"

# Create a list of files
files = [
    {
        'type': 'image',
        'bytes': image_bytes,
        'format': 'jpeg'
    },
    # Uncomment when you have a document to test with
    # {
    #     'type': 'document',
    #     'bytes': document_bytes,
    #     'name': 'report.pdf',
    #     'format': 'pdf'
    # }
]

# Create a message with multiple file types
multi_file_message = llm_manager.create_multi_file_message(
    files=files,
    text="Please analyze this content."
)

# Send the message to the model
# Note: This will only work if your model supports the file types you've included
# response = llm_manager.converse(messages=[multi_file_message])
# print(response.get_content_text())

## Using the Builder Pattern for Complex Messages

The ContentBuilder class provides a fluent interface for creating complex multimodal messages:

In [ ]:
# Create a new builder
builder = llm_manager.create_content_builder()

# Add content in a fluent chain
message = builder \
    .add_text("I have a question about this image:") \
    .add_image(image_bytes=image_bytes, image_format="jpeg") \
    .add_text("What is this structure, and when was it built?") \
    .build()

# Send the message to the model
response = llm_manager.converse(messages=[message])
print(response.get_content_text())

## S3 Support

The ContentBuilder also supports referencing files in S3:

In [ ]:
# This example shows the syntax but won't run without valid S3 references
# builder = llm_manager.create_content_builder()
# message = builder \
#     .add_text("Please analyze these documents:") \
#     .add_image(
#         s3_bucket="my-bucket",
#         s3_key="images/chart.png",
#         image_format="png"
#     ) \
#     .add_document(
#         document_name="quarterly_report.pdf",
#         document_format="pdf",
#         s3_bucket="my-bucket",
#         s3_key="documents/quarterly_report.pdf"
#     ) \
#     .build()

# response = llm_manager.converse(messages=[message])
# print(response.get_content_text())

## Conclusion

This notebook demonstrates the enhanced multimodal content capabilities of the LLMManager class. Key features include:

1. Support for text, images, documents, and videos in a single message
2. Multiple ways to provide content (bytes, S3, URI)
3. A flexible builder pattern for creating complex messages
4. Support for various file formats and sources

These capabilities make it easier to build rich multimodal applications with AWS Bedrock's Converse API.